# Imports

In [ ]:
import os, sys

import numpy as np

import torch

import time

# For loops

In [ ]:
def forloopdists(feats,protos):

  dist=np.empty((feats.shape[0],protos.shape[0]))
  for i in range(feats.shape[0]):
    for k in range(protos.shape[0]):
      dist[i,k]=  np.dot( feats[i,:]-protos[k,:],(feats[i,:]-protos[k,:]).transpose()) # dot product of a vector with itself is the square of the norm of that vector

  return dist

# Numpy

In [ ]:
def numpydists(feats,protos):
  dist0 = -2 *np.matmul(feats,protos.transpose())
  print(dist0.shape)
  dist1 = np.sum( feats**2,axis=1 )#[:,np.newaxis]
  print(dist1.shape)
  dist2 = np.sum( protos**2,axis=1 )#[np.newaxis,:]
  print(dist2.shape)

  dist = dist0 + dist1[:,np.newaxis] + dist2[np.newaxis,:]

  return dist

# Pytorch

In [ ]:
def pytorchdists(feats, protos):
  
  print(feats.size(),protos.size())

  print(feats.device)

  #dist= torch.sum(torch.pow(ft.unsqueeze(1)-prot.unsqueeze(0),2.0),dim=2) #N,P,D
  dist= -2*torch.mm(feats,protos.t()) 
  dist+= torch.sum(torch.pow(feats,2.0),dim=1).unsqueeze(1) 
  dist+= torch.sum(torch.pow(protos,2.0),dim=1).unsqueeze(0) 

  return dist.cpu().numpy()

# Run everything

In [ ]:
########
##
## if you have less than 8 gbyte, then reduce from 250k
##
###############
feats = np.random.normal(size=(250000,300)) #5000 instead of 250k for forloopdists
protos = np.random.normal(size=(500,300))

# since = time.time()
# dists0=forloopdists(feats,protos)
# time_elapsed=float(time.time()) - float(since)
# print('For loops; Comp complete in {:.3f}s'.format( time_elapsed ))

since = time.time()
dists1 = numpydists(feats,protos)
time_elapsed = float(time.time()) - float(since)

print('Numpy; Comp complete in {:.3f}s'.format( time_elapsed ))
print(dists1.shape)

device = torch.device('cpu')
feats = torch.from_numpy(feats).to(device)
protos = torch.from_numpy(protos).to(device)
since = time.time()
dists2 = pytorchdists(feats, protos)
time_elapsed = float(time.time()) - float(since)

print('PyTorch CPU; Comp complete in {:.3f}s'.format( time_elapsed ))
print(dists2.shape)

device = torch.device('cuda:0')
feats = feats.to(device)
protos = protos.to(device)

since = time.time()
dists3 = pytorchdists(feats, protos)
time_elapsed = float(time.time()) - float(since)

print('PyTorch GPU; Comp complete in {:.3f}s'.format( time_elapsed ))
print(dists3.shape)

print('df',np.max(np.abs(dists1-dists2)))


(250000, 500)
(250000,)
(500,)
Numpy; Comp complete in 2.677s
(250000, 500)
torch.Size([250000, 300]) torch.Size([500, 300])
cpu
PyTorch CPU; Comp complete in 1.762s
(250000, 500)
torch.Size([250000, 300]) torch.Size([500, 300])
cuda:0
PyTorch GPU; Comp complete in 0.695s
(250000, 500)
df 3.410605131648481e-13


## exp

In [ ]:
import numpy as np
import time

feats=np.random.normal(size=(2500,300))
protos=np.random.normal(size=(500,300))

since = time.time()
dist0 = -2 *np.matmul(feats,protos.transpose())
dist1 = np.sum( feats**2,axis=1 )#[:,np.newaxis]
dist2 = np.sum( protos**2,axis=1 )#[np.newaxis,:]
disti = dist0 + dist1[:,np.newaxis] + dist2[np.newaxis,:]
time_elapsed=float(time.time()) - float(since)

print('Numpy1; Comp complete in {:.3f}s'.format( time_elapsed ))


print(disti.shape)

# since = time.time()
# distj = np.sum(np.power(feats[:, np.newaxis] - protos[np.newaxis, :], 2), axis=2) # N,P,D
# time_elapsed=float(time.time()) - float(since)


print('Numpy2; Comp complete in {:.3f}s'.format( time_elapsed ))

print(distj.shape)

since = time.time()
distj = sum(np.power(a[:,np.newaxis]-b, 2) for a, b in zip(feats.T, protos.T))
time_elapsed=float(time.time()) - float(since)


print('native; Comp complete in {:.3f}s'.format( time_elapsed ))



print('df',np.max(np.abs(disti-distj)))


Numpy1; Comp complete in 0.029s
(2500, 500)
Numpy2; Comp complete in 0.029s
(300, 300)
native; Comp complete in 11.410s
df 1.5916157281026244e-12


In [ ]:
import torch
import numpy as np

feats=np.random.normal(size=(2500,300))
protos=np.random.normal(size=(500,300))

feats=torch.from_numpy(feats)
protos=torch.from_numpy(protos)

since = time.time()
dist= -2*torch.mm(feats,protos.t()) 
dist+= torch.sum(torch.pow(feats, 2.0),dim=1).unsqueeze(1) 
dist+= torch.sum(torch.pow(protos, 2.0),dim=1).unsqueeze(0) 
time_elapsed=float(time.time()) - float(since)

print('Torch1; Comp complete in {:.3f}s'.format( time_elapsed ))


print(dist.shape)

since = time.time()
distj = torch.sum(torch.pow(feats.unsqueeze(1)-protos.unsqueeze(0),2.0),dim=2) #N,P,D
time_elapsed=float(time.time()) - float(since)

print('Torch2; Comp complete in {:.3f}s'.format( time_elapsed ))

print(distj.shape)

print('df', torch.max(torch.abs(dist-distj)))


Torch1; Comp complete in 0.025s
torch.Size([2500, 500])
Torch2; Comp complete in 2.300s
torch.Size([2500, 500])
df tensor(3.4106e-13, dtype=torch.float64)
